<a href="https://colab.research.google.com/github/divyansh070/paper_implemtaions/blob/main/transformer_testing/bigram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

learning from https://youtu.be/UU1WVnMk4E8?feature=shared vedio **Create a Large Language Model from Scratch with Python – Tutorial**

In [79]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [80]:
import torch

In [81]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

cpu


In [82]:
with open("/content/wizard_of_oz.txt", "r", encoding="utf-8") as f:
    text = f.read()
# print(len(text))
chars=sorted(set(text))
print(len(chars))
print(chars)
vocab_size=len(chars)

92
['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”', '•', '™', '\ufeff']


In [83]:
string_to_int={ch:i for i,ch in enumerate(chars)}
int_to_string={i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s ]
decode = lambda l: "".join([int_to_string[i] for i in l])

In [84]:
data = torch.tensor(encode(text),dtype=torch.long)
print(data[0:100])

tensor([91, 48, 65, 62,  1, 44, 75, 72, 67, 62, 60, 77,  1, 35, 78, 77, 62, 71,
        59, 62, 75, 64,  1, 62, 30, 72, 72, 68,  1, 72, 63,  1, 32, 72, 75, 72,
        77, 65, 82,  1, 58, 71, 61,  1, 77, 65, 62,  1, 51, 66, 83, 58, 75, 61,
         1, 66, 71,  1, 43, 83,  0,  1,  1,  1,  1,  0, 48, 65, 66, 76,  1, 62,
        59, 72, 72, 68,  1, 66, 76,  1, 63, 72, 75,  1, 77, 65, 62,  1, 78, 76,
        62,  1, 72, 63,  1, 58, 71, 82, 72, 71])


In [85]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[71, 64, 58, 59, 72, 72, 76,  1],
        [58, 71, 61,  1, 58,  1, 71, 62],
        [ 3, 51, 65, 58, 77,  1, 64, 72],
        [77, 12,  3,  1, 60, 72, 71, 77]])
targets:
tensor([[64, 58, 59, 72, 72, 76,  1, 60],
        [71, 61,  1, 58,  1, 71, 62, 66],
        [51, 65, 58, 77,  1, 64, 72, 72],
        [12,  3,  1, 60, 72, 71, 77, 66]])


In [86]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


In [87]:
import torch
import torch.nn as nn

token = torch.tensor([1, 2, 3, 4, 5, 6])  # token indices
embedding = nn.Embedding(num_embeddings=7, embedding_dim=3)  # vocab size = 10, dim = 3

output = embedding(token)  # shape: (6, 3)
print(output)

tensor([[-0.1561,  0.0089,  0.2885],
        [-1.5089,  1.7304,  0.0097],
        [ 0.5964,  0.5623,  0.1016],
        [-0.3619,  0.5524,  1.1342],
        [-0.9880, -0.6850,  0.5136],
        [ 2.1052, -0.7911,  0.0927]], grad_fn=<EmbeddingBackward0>)


In [88]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)


        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

In [89]:

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


!O2/JvzUx[rwu‘RiGh0 Ivp%3#—UK4jiUC”™b1”3SZ﻿on
S9OT ,p‘/#VO.p[?ApdjMDv™.K$CGm?’kTWP1#6_:!Xub5WlQG!f2b7yK)"9P!/gUn
P*e)Gsv]tMlI6bi﻿oaM.g!•™xy3,•A$oLT‘vk4lUEQR%ElXt.“y—xbReO5G[?﻿mA0‘F8]g'Zwf—Lp/5]o‘$!2.1MzKtyz“M1‘QGA—J'f—xU_Vsmfq•AEgiVyGu22ZkspoQu1hMw"LjR#K0g1w&?—#_pP!erG tUM#FRXS(odAp™xF%Nt-e#l*uIR•rSAs$IW,“MZ7rhR 1R:H7/G"•$DJxJ—jP!1*b‘u)]gbUgu‘—])_d2&Lp]VB[1.A5“6Cal0&Ey)﻿R•l59bUZh:ON™ lXIOG22J't‘—uZJiqGK_11HC%usPxhd8_y]2SWrg'yB[
noQC(-K$XUxSTss:oQ8wc*']nOO2_&r1,M5lFInO,Mh)xc[sQm5)L?5JH22EN‘P?A4’L


In [90]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.910, val loss: 4.940
step: 250, train loss: 4.856, val loss: 4.908
step: 500, train loss: 4.789, val loss: 4.860
step: 750, train loss: 4.717, val loss: 4.794
4.462522983551025


In [91]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


h—&n”5_wdf—tn﻿ijvxShuD”(mo&Los”k/:OMIdZP7dAn6,d?tP*a4yh6f—tVaFQ$IoRIFan’#“tV:oI?nzk;Xy﻿uf-Eu)•”g'ndA0c—’srAL3, IA3h﻿nq;PooQG!”™Lo.“NZ’9u6)Z.:-Q’K/Sc[5O&M[4JVBBpel8]vknXhhYjUjOf
&‘zHD•[A&Z("a••AAAm_n6Xck—™b‘
ZzLj"’x2UnD4xA5U
”jl5PV[h7KP™LYY—”(FDnOTYdApnl;/: Mh?L"Pk;”(Zv”&_d2 x_ra4z"’•z,I*H[UZb“qjZKg%e”wOfRR7‘—%D ,O.﻿J%'PWF,xHneZ2a7;gr_rS;Ygz•7X8da.﻿i/jZpksrVDmY5Z?!9.OR:OAr.pK)].#qvfu%H[#;,Kg'!AD2cXdf)_7;t1H[D78﻿L8—tOW!p&•tMpVQG,9“﻿$uf9 
2
bOjZ.C’F81*#WApT_V%?vj#"zbGuImuclBcY88Sw™ia?Dr.8™ Q/ODb_z1


In [92]:
!git clone https://github.com/divyansh070/paper_implemtaions.git
%cd paper_implemtaions

Cloning into 'paper_implemtaions'...
/content/paper_implemtaions
